Updated on Tues Aug 10 12:56 2021

PF code which fits an ellipse around a PF and then identifies TRMM features within the larger ellipse
Based on code from Weixin Xu, Kyle Chudler, Brody Fuchs, Zach Bruick, and Marqi Rocque

Current outputs are PF location (center of ellipse), area, conv/strat areas, mean dBZ
at each level, max dBZ at each level, and echo top height.


For 1km runs with CRSIM new data

In [1]:
import numpy as np  
from scipy.ndimage import label, generate_binary_structure, center_of_mass
import matplotlib.patches as patches
from numpy.linalg import eig
from wrf_convstrat import conv_strat_latlon
import wrf
import pyart
from matplotlib.lines import Line2D
#############

#takes in x and y points and returns parameters which define a ellipse that surrounds these points
#(Medioni et al. 2000; Nesbitt et al. 2006)
def get_ellipse(xs,ys):
    npts = len(xs)
    i11 = np.sum(ys**2) / npts
    i22 = np.sum(xs**2) / npts
    i12 = -np.sum(xs * ys) / npts
    tensor = [ [i11, i12], [i12,i22] ]
    eig_vals, eig_vecs = eig(tensor)
    semimajor = np.sqrt(eig_vals[0]) * 2.0
    semiminor = np.sqrt(eig_vals[1]) * 2.0
    major = semimajor * 2.0
    minor = semiminor * 2.0
    eig_vecs = eig_vecs[:,0]
    orientation = np.math.atan2(eig_vecs[1], eig_vecs[0]) - np.deg2rad(90.0)
    
    return major, minor, orientation


def PF_finder(refl, rr, lon, lat, z, w, t, dx = 3, thresh_dbz = 0, min_conv_n = 20, min_strat_n = 20, rainrate_array = None):
    
    x1 = np.arange(0, np.shape(lat)[1], 1)
    y1 = np.arange(0, np.shape(lat)[0], 1)
    
    z1 = np.arange(500, 20001, 500)
    refl = np.array(wrf.interplevel(refl, z, z1))
    w_int = np.array(wrf.interplevel(w, z, z1))
    temp_int = np.array(wrf.interplevel(t, z, z1))

    #calculate convective/stratiform area at 2km AGL
    #returns an array where 0 = stratiform, 1-5 = convection (depending on dbz threshold)
    
    # 3 = 2 km, 4 = 2.5 km
    dbz_2km = refl[4,:,:]
    cs,cc,bkgnd = conv_strat_latlon(dbz_2km, lat, lon)
    
    cs[np.where(dbz_2km < 0)] = -1
    cs[np.where(np.isnan(dbz_2km))] = -1

    strat_inds = cs == 0
    conv_inds  = cs > 0
        
    label_mask = generate_binary_structure(2,2)
    x,y = np.meshgrid(x1,y1)
    
    max_dbz = np.zeros([lon.shape[0],lon.shape[1]])
    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            max_dbz[i,j] = np.nanmax(refl[:,i,j])

            
    conv_ind_40 = np.zeros([lon.shape[0],lon.shape[1]])
    
    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            if max_dbz[i,j] >= 40 and conv_inds[i,j] == 1:
                conv_ind_40[i,j] = 1
            else:
                conv_ind_40[i,j] = 0
    
    label_mask3 = generate_binary_structure(3,3)

    w5_ind = np.zeros([z1.shape[0],lon.shape[0],lon.shape[1]])
    
    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            for k in range(z1.shape[0]):
                #if temp_int[k,i,j] >= -40 and temp_int[k,i,j] <= -5 and w_int[k,i,j] >= 5:
                if w_int[k,i,j] >=5:
                    w5_ind[k,i,j] = 1
                else:
                    w5_ind[k,i,j] = 0
                
    w10_ind = np.zeros([z1.shape[0],lon.shape[0],lon.shape[1]])
        
    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            for k in range(z1.shape[0]):
                if temp_int[k,i,j] >= -40 and temp_int[k,i,j] <= -5 and w_int[k,i,j] >= 10:
                    w10_ind[k,i,j] = 1
                else:
                    w10_ind[k,i,j] = 0
        
        
    pf_groups, n_groups = label(dbz_2km > thresh_dbz, structure = label_mask)
    pf_groups_conv, n_groups_conv = label(conv_ind_40, structure = label_mask)
    pf_groups_strat, n_groups_strat = label(strat_inds, structure = label_mask)
    pf_groups_up5, n_groups_up5 = label(w5_ind, structure = label_mask3)

#subtract 1 from pf_groups so that the indexing works out nicer

    pf_groups -= 1
    pf_groups_conv -= 1
    pf_groups_strat -= 1
    pf_groups_up5 -= 1
            
    pf_locs = center_of_mass(dbz_2km >= thresh_dbz, pf_groups , np.arange(n_groups))
    pf_locs = [(x[int(l[0]),int(l[1])], y[int(l[0]),int(l[1])]) for l in pf_locs]
        
    pf_locs_conv = center_of_mass(conv_ind_40, pf_groups_conv , np.arange(n_groups_conv))
    pf_locs_conv = [(x[int(l[0]),int(l[1])], y[int(l[0]),int(l[1])]) for l in pf_locs_conv]
    
    pf_locs_strat = center_of_mass(strat_inds, pf_groups_strat , np.arange(n_groups_strat))
    pf_locs_strat = [(x[int(l[0]),int(l[1])], y[int(l[0]),int(l[1])]) for l in pf_locs_strat]   
    
    pf_locs_up5 = center_of_mass(w5_ind, pf_groups_up5 , np.arange(n_groups_up5))
    y2,z2,x2 = np.meshgrid(y1,z1,x1)
    pf_locs_up5 = [(z2[int(l[0]), int(l[1]), int(l[2])], y2[int(l[0]),int(l[1]),int(l[2])], x2[int(l[0]),int(l[1]),int(l[2])]) for l in pf_locs_up5] 
    
    
    ellipses           = []
    area               = []
    conv_area          = []
    strat_area         = []
    mean_refl_by_alt   = [] 
    max_refl_by_alt    = []
    echo_top           = []
    echo_top_40        = []
    wmax               = []
    wmax_alt           = []
    w5_vol             = []
    w10_vol            = []
    mean_rr_wrf        = []
    max_rr_wrf         = []
    mean_conv_rr_wrf   = []
    mean_strat_rr_wrf  = []
    mean_rr_zr         = []
    max_rr_zr          = []
    mean_conv_rr_zr    = []
    mean_strat_rr_zr   = []
    
    dwcc_ellipses = [[] for i in range(n_groups)]
    dwcc_area = [[] for i in range(n_groups)]
    dwcc_locx = [[] for i in range(n_groups)]
    dwcc_locy = [[] for i in range(n_groups)]
    dwcc_maxdbz_10km = [[] for i in range(n_groups)]
    up_dwcc_max = [[] for i in range(n_groups)]
    up_dwcc_mean = [[] for i in range(n_groups)]

    wcc_ellipses = [[] for i in range(n_groups)]
    wcc_area = [[] for i in range(n_groups)]
    wcc_locx = [[] for i in range(n_groups)]
    wcc_locy = [[] for i in range(n_groups)]
    up_wcc_max = [[] for i in range(n_groups)]
    up_wcc_mean = [[] for i in range(n_groups)]

    dcc_ellipses = [[] for i in range(n_groups)]
    dcc_area = [[] for i in range(n_groups)]
    dcc_locx = [[] for i in range(n_groups)]
    dcc_locy = [[] for i in range(n_groups)]
    dcc_maxdbz_10km = [[] for i in range(n_groups)]
    up_dcc_max = [[] for i in range(n_groups)]
    up_dcc_mean = [[] for i in range(n_groups)]

    bsr_ellipses = [[] for i in range(n_groups)]
    bsr_area = [[] for i in range(n_groups)]
    bsr_locx = [[] for i in range(n_groups)]
    bsr_locy = [[] for i in range(n_groups)]
    
    up5_locx = [[] for i in range(n_groups)]
    up5_locy = [[] for i in range(n_groups)]
    up5_locz = [[] for i in range(n_groups)]
    up5_pf_inds = [[] for i in range(n_groups)]
    up5_vol = [[] for i in range(n_groups)]
    up5_minz = [[] for i in range(n_groups)]
    up5_maxz = [[] for i in range(n_groups)]
    up5_depth = [[] for i in range(n_groups)]
    up5_maxspd = [[] for i in range(n_groups)]
    up5_meanspd = [[] for i in range(n_groups)]
    up5max_alt = [[] for i in range(n_groups)]

    for group_num in np.arange(n_groups):
        
        pf_inds = pf_groups == group_num
            
        pf_xs = x[pf_inds] - pf_locs[group_num][0]
        pf_ys = y[pf_inds] - pf_locs[group_num][1]
                    
        major, minor, orientation = get_ellipse(pf_xs, pf_ys)
        
        ellipses.append(patches.Ellipse(pf_locs[group_num], width = major, height = minor, 
                                    angle = np.rad2deg(orientation), facecolor = 'None', edgecolor = 'k', lw = 2))
        
        npix = np.sum(pf_inds)
        n_conv_pix  = np.sum((pf_inds) & (conv_inds))
        n_strat_pix  = np.sum((pf_inds) & (strat_inds))
    
        area.append(npix*dx**2)
        conv_area.append(n_conv_pix*dx**2)
        strat_area.append(n_strat_pix*dx**2)
        
        mean_refl_by_alt_pf = np.zeros(z1.shape)
        max_refl_by_alt_pf  = np.zeros(z1.shape)
        for zi in range(len(z1)):
            rain_inds = refl[zi, pf_inds] > 0
            if np.sum(rain_inds) > 0:
                mean_refl_by_alt_pf[zi] = np.mean(refl[zi,pf_inds][rain_inds])
                max_refl_by_alt_pf[zi]  = np.max(refl[zi,pf_inds][rain_inds])
        
        mean_refl_by_alt.append(mean_refl_by_alt_pf)
        max_refl_by_alt.append(max_refl_by_alt_pf)

        echo_top_ind = len(max_refl_by_alt_pf) - np.argmax((max_refl_by_alt_pf > 0)[::-1]) - 1
        echo_top.append(z1[echo_top_ind])
        
        echo_top_40_ind = len(max_refl_by_alt_pf) - np.argmax((max_refl_by_alt_pf > 40)[::-1]) - 1
        
        if echo_top_40_ind > echo_top_ind:
            echo_top_40.append(0)
        else:
            echo_top_40.append(z1[echo_top_40_ind])

          
        all_ws = w_int[:,pf_inds]
        wmax.append(np.nanmax(all_ws))
        max_index = np.argmax(all_ws == np.nanmax(all_ws), axis=0)
        tallest_max_index = np.max(max_index)
        wmax_alt.append(z1[tallest_max_index])
        
                
        #w5_ind = all_ws >= 5
        #tmax_ind = all_temps <= -5
        #tmin_ind = all_temps >= -40
                
        w5_vol.append(np.sum(w5_ind[:,pf_inds])*dx*dx*0.5)
        
        #w10_ind = all_ws >= 10
        w10_vol.append(np.sum(w10_ind[:,pf_inds])*dx*dx*0.5)
        

        # try to add in rain rate, calculate accumulated rain at that time before here:
        
        mean_rr_wrf.append(np.mean(rr[pf_inds]))
        max_rr_wrf.append(np.max(rr[pf_inds]))
        mean_conv_rr_wrf.append(np.mean(rr[(pf_inds) & (conv_inds)]))
        mean_strat_rr_wrf.append(np.mean(rr[(pf_inds) & (strat_inds)]))
        
        #from DROPS2.0 (Chen et al. 2017; Eq3a)
        
        ze = 10**(dbz_2km/10)
        rr_zr = 0.02*(ze**0.657)
        mean_rr_zr.append(np.mean(rr_zr[pf_inds]))
        max_rr_zr.append(np.max(rr_zr[pf_inds]))
        mean_conv_rr_zr.append(np.mean(rr_zr[(pf_inds) & (conv_inds)]))
        mean_strat_rr_zr.append(np.mean(rr_zr[(pf_inds) & (strat_inds)]))
        
        
        #look for TRMM features within the larger feature
        for conv_group_num in np.arange(n_groups_conv):
            if np.sum((pf_inds) & (pf_groups_conv == conv_group_num)) > min_conv_n/dx**2:

                inds = (pf_inds) & (pf_groups_conv == conv_group_num)
                z_10km = np.where(z1 == 10000)[0][0]
                tall_inds_10km = refl[z_10km, inds] > 40
                
                if np.sum(tall_inds_10km) > 0:
                    print ('DWCC found')

                    pf_xs = x[inds] - pf_locs_conv[conv_group_num][0]
                    pf_ys = y[inds] - pf_locs_conv[conv_group_num][1]
                    
                    major, minor, orientation = get_ellipse(pf_xs, pf_ys)
                    
                    dwcc_ellipses[group_num].append(patches.Ellipse(pf_locs_conv[conv_group_num],width = major, height = minor, 
                                      angle = np.rad2deg(orientation), facecolor = 'None', edgecolor = 'g', lw = 2))
                    dwcc_area[group_num].append(np.sum(inds)*dx**2)
                    
                    dwcc_locx[group_num].append(pf_locs_conv[conv_group_num][0])
                    dwcc_locy[group_num].append(pf_locs_conv[conv_group_num][1])
                    
                    dwcc_maxdbz_10km[group_num].append(np.max(refl[z_10km, inds]))
                    
                    dwcc_ws = w_int[:,inds]
                    up_dwcc_max[group_num].append(np.nanmax(dwcc_ws))
                    dwcc_ws = w_int[:, inds]
                    up_dwcc_mean[group_num].append(np.nanmean(dwcc_ws))
                                       
                else:
                    print ('WCC found')
                    
                    pf_xs = x[inds] - pf_locs_conv[conv_group_num][0]
                    pf_ys = y[inds] - pf_locs_conv[conv_group_num][1]
                    
                    major, minor, orientation = get_ellipse(pf_xs, pf_ys)
                    
                    wcc_ellipses[group_num].append(patches.Ellipse(pf_locs_conv[conv_group_num],width = major, height = minor, 
                                      angle = np.rad2deg(orientation), facecolor = 'None', edgecolor = 'b', lw = 2))
                    wcc_area[group_num].append(np.sum(inds)*dx**2)
                    wcc_locx[group_num].append(pf_xs)
                    wcc_locy[group_num].append(pf_ys)
                    
                    wcc_ws = w_int[:,inds]
                    up_wcc_max[group_num].append(np.nanmax(wcc_ws))
                    wcc_ws = w_int[:, inds]
                    up_wcc_mean[group_num].append(np.nanmean(wcc_ws))

            else:
                
                inds = (pf_inds) & (pf_groups_conv == conv_group_num)

                z_10km = np.where(z1 == 10000)[0][0]
                tall_inds_10km = refl[z_10km, inds] > 40
                
                if np.sum(tall_inds_10km) > 0:
                    print ('DCC found')
                    
                    pf_xs = x[inds] - pf_locs_conv[conv_group_num][0]
                    pf_ys = y[inds] - pf_locs_conv[conv_group_num][1]
                    
                    major, minor, orientation = get_ellipse(pf_xs, pf_ys)
                    
                    dcc_ellipses[group_num].append(patches.Ellipse(pf_locs_conv[conv_group_num],width = major, height = minor, 
                                      angle = np.rad2deg(orientation), facecolor = 'None', edgecolor = 'r', lw = 2))
                    dcc_area[group_num].append(np.sum(inds)*dx**2)
                    dcc_locx[group_num].append(pf_xs)
                    dcc_locy[group_num].append(pf_ys)
                    
                    dcc_maxdbz_10km[group_num].append(np.max(refl[z_10km, inds]))
                    
                    dcc_ws = w_int[:,inds]
                    up_dcc_max[group_num].append(np.nanmax(dcc_ws))
                    dcc_ws = w_int[:, inds]
                    up_dcc_mean[group_num].append(dcc_ws[dcc_ws>0].mean())


                                   
        #identify BSR       
        for strat_group_num in np.arange(n_groups_strat):
            if np.sum((pf_inds) & (pf_groups_strat == strat_group_num)) > min_strat_n/dx**2:
                print ('BSR found')

                inds = (pf_inds) & (pf_groups_strat == strat_group_num)
                
                pf_xs = x[inds] - pf_locs_strat[strat_group_num][0]
                pf_ys = y[inds] - pf_locs_strat[strat_group_num][1]
                
                major, minor, orientation = get_ellipse(pf_xs, pf_ys)
                
                bsr_ellipses[group_num].append(patches.Ellipse(pf_locs_strat[strat_group_num],width = major, height = minor, 
                                  angle = np.rad2deg(orientation), facecolor = 'None', edgecolor = 'gold', lw = 2))
                bsr_area[group_num].append(np.sum(inds)*dx**2)
                bsr_locx[group_num].append(pf_xs)
                bsr_locy[group_num].append(pf_ys)
              
            
        if npix*dx**2 > 5000: #set an area theshold on pf when getting updrafts to save time

            for up5_group_num in np.arange(n_groups_up5):
                inds = (pf_inds) & (pf_groups_up5 == up5_group_num)
                npix = np.sum(inds)
                
                if npix*dx*dx*0.5 > 100: #set a volume threshold on updrafts to save time
                    up5_pf_inds[group_num].append(inds)
                                    
                    pf_xs = pf_locs_up5[up5_group_num][2]
                    pf_ys = pf_locs_up5[up5_group_num][1]
                    pf_zs = pf_locs_up5[up5_group_num][0]
                    
                    up5_locx[group_num].append(pf_xs)
                    up5_locy[group_num].append(pf_ys)
                    up5_locz[group_num].append(pf_zs)
                    
                    zs = z2[inds]
                    zmin = np.min(zs)
                    zmax = np.max(zs)
                    depth = zmax-zmin

                    up5_minz[group_num].append(zmin)
                    up5_maxz[group_num].append(zmax)
                    up5_depth[group_num].append(depth)

                    up5_vol[group_num].append(npix*dx*dx*0.5)
                    
                    up5_maxspd[group_num].append(np.nanmax(w_int[inds]))
                    up5_meanspd[group_num].append(np.nanmean(w_int[inds]))
                    
                    max_index = np.argmax(w_int[inds] == np.nanmax(w_int[inds]), axis=0)
                    tallest_max_index = np.max(max_index)
                    up5max_alt[group_num].append(zs[tallest_max_index])
                        

    return ellipses, np.array(pf_locs)[:,0], np.array(pf_locs)[:,1], area, conv_area, strat_area, mean_refl_by_alt, max_refl_by_alt, echo_top, echo_top_40, wmax, wmax_alt, w5_vol, w10_vol, mean_rr_wrf, max_rr_wrf, mean_conv_rr_wrf, mean_strat_rr_wrf, mean_rr_zr, max_rr_zr, mean_conv_rr_zr, mean_strat_rr_zr, dcc_ellipses, dcc_area, dcc_locx, dcc_locy, dcc_maxdbz_10km, up_dcc_max, up_dcc_mean, dwcc_ellipses, dwcc_area,dwcc_locx, dwcc_locy, dwcc_maxdbz_10km, up_dwcc_max, up_dwcc_mean, dwcc_ellipses, wcc_area,wcc_locx, wcc_locy, up_wcc_max, up_wcc_mean, bsr_ellipses, bsr_area, bsr_locx, bsr_locy, up5_locx, up5_locy, up5_locz, up5_vol, up5_minz, up5_maxz, up5_depth, up5_maxspd, up5_meanspd, up5max_alt


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [29]:
def draw_ellipses(date, reflect, rr, lon, lat, z, ter, ell, dcc_ell, dwcc_ell, wcc_ell, bsr_ell, up5_x, up5_y):
    
    new_refl = wrf.interplevel(reflect, z, 2500)
    max_dbz = np.zeros([lon.shape[0],lon.shape[1]])
    for i in range(lon.shape[0]):
        for j in range(lon.shape[1]):
            max_dbz[i,j] = np.nanmax(reflect[:,i,j])
            
    
    plt.figure(figsize=(10,8))

    cs1 = plt.pcolormesh(new_refl, cmap='pyart_HomeyerRainbow', vmin=0, vmax=70)
    #cs1 = plt.pcolormesh(rr, cmap='YlGnBu', vmin=0.001, vmax=25)
    cs1.cmap.set_under('w')
    plt.contour(ter, levels=[500], colors='k', linewidths=0.5)
    
    #plt.contour(max_dbz, levels=[40], colors='magenta', linewidths=1)
    
    ax = plt.gca()
    for e in ell:
        ax.add_patch(e)
            
    for e in dcc_ell:
        for f in e:
            ax.add_patch(f)
            
    for e in dwcc_ell:
        for f in e:
            ax.add_patch(f)
            
    for e in wcc_ell:
        for f in e:
            ax.add_patch(f)
            
    for e in bsr_ell:
        for f in e:
            ax.add_patch(f)
            
#     for e in up5_ell:
#         for f in e:
#             ax.add_patch(f)
        
    ax.grid()
    ax.set_xlim(0, 702)
    ax.set_ylim(0, 897)
    
    custom_lines = [Line2D([0], [0], color='k', lw=2),
                    Line2D([0], [0], color='r', lw=2),
                    Line2D([0], [0], color='g', lw=2),
                    Line2D([0], [0], color='b', lw=2),
                    Line2D([0], [0], color='orange', lw=2),
                    Line2D([0], [0], color='k', lw=0.001, marker='x', markerfacecolor='k', markersize=10)]
    ax.legend(custom_lines, ['PF', 'DCC', 'DWCC', 'WCC', 'BSR', 'w5'], loc=1)
              #loc='center left', bbox_to_anchor=(1, 0.5))
    
    #chivo location = [391, 509] 3km
    #chivo location = [316, 511] 1km
    
    #lons = (lon[up5_y[68],up5_x[68]])
    #lats = (lat[up5_y[68],up5_x[68]])
    plt.scatter(up5_x[68], up5_y[68], color='k', s=50, marker='x', zorder=10)
    
    plt.text(316-xmin, 511-ymin, 'C', fontsize=14, fontweight='semibold')
    
    
    cbar = plt.colorbar(cs1)
    cbar.ax.set_title('[dBZ]')
    plt.title('RELAMPAGO 1 km CTRL '+date+'\nPF ID testing\n(CR-SIM Reflectivity 2.5 km MSL with ellipses and updrafts)')
    #plt.title('RELAMPAGO CTRL '+date+'\nPF ID testing\n(Rain Rate from RAINNC [mm/10min] with ellipses)')
    #plt.show()
    plt.savefig('wrf_relampago_ctrl_1km_crsim_'+date+'_trmm_ellipses_updraft.png', dpi=400)#, bbox_inches='tight')
    plt.close()

##############################################################################


/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
from netCDF4 import Dataset
from datetime import datetime
import matplotlib.pyplot as plt

path = '/rasmussen-scratch/mrocque/research/relampago/wrfout_copy/'
file = 'wrfout_d01_2018-12-14_03:01:00'
data = Dataset(path+file, 'r')
#print (data.variables.keys())

xmin = 0
xmax = 702
ymin = 0
ymax = 897

n=0
lon = np.array(data.variables['XLONG'][n,ymin:ymax,xmin:xmax])
lat = np.array(data.variables['XLAT'][n,ymin:ymax,xmin:xmax])

reflect = np.array(data.variables['REFL_10CM'][n,:,ymin:ymax,xmin:xmax])
reflect = np.array(data.variables['crsim_Zhh'][n,:,ymin:ymax,xmin:xmax])
#reflect = np.array(wrf.getvar(data, 'dbz', timeidx=n)[:,ymin:ymax,xmin:xmax])

z = np.array(wrf.getvar(data, 'z', units='m', msl=True, timeidx=n)[:,ymin:ymax,xmin:xmax])

w_raw = np.array(data.variables['crsim_DV90'][n,:,ymin:ymax,xmin:xmax])
#w_raw = np.array(wrf.getvar(data, 'wa', units='m s-1', timeidx=n)[:,ymin:ymax,xmin:xmax])
temp_raw = np.array(wrf.getvar(data, 'temp', units='degC', timeidx=n)[:,ymin:ymax,xmin:xmax])

ter = wrf.getvar(data, 'ter', units='m', timeidx=n)[ymin:ymax,xmin:xmax]

times =  (wrf.extract_times(data, None))
newt = []
for t in np.arange(0, len(times), 1):
    newt.append(datetime.utcfromtimestamp(times[t].tolist()/1e9))
date = newt[n].strftime('%Y%m%d_%H%M')
print (date)

rain = np.array(data.variables['RAINNC'][n,ymin:ymax,xmin:xmax])
path2 = '/rasmussen-scratch/mrocque/research/relampago/wrf_ctrl_1km/'
file2 = 'wrfout_d01_2018-12-14_02:51:00'
datai = Dataset(path2+file2, 'r')
raini = np.array(datai.variables['RAINNC'][0,ymin:ymax,xmin:xmax])
rrate = rain - raini

ell, locx, locy, pf_area, carea, sarea, mean_dbz_z, max_dbz_z, et, et40, upmax, upmax_alt, \
    upvol5, upvol10, rr_mean_wrf, rr_max_wrf, crr_mean_wrf, srr_mean_wrf, rr_mean_zr, rr_max_zr, crr_mean_zr, srr_mean_zr, dcc_ell, dcc_ar, \
    dcc_x, dcc_y, dcc_10km, dcc_wmax, dcc_wmean, dwcc_ell, dwcc_ar, dwcc_x, dwcc_y, dwcc_10km, dwcc_wmax, dwcc_wmean, wcc_ell, wcc_ar, \
    wcc_x, wcc_y, wcc_wmax, wcc_wmean, bsr_ell, bsr_ar, bsr_x, bsr_y, up5_x, up5_y, up5_z, upvol5_ind, minzup5, maxzup5, depthup5, maxspdup5, meanspdup5, maxaltup5  = PF_finder(reflect, rrate, lon, lat, z, w_raw, temp_raw, 1, 0, 1000, 40000, None)



/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


20181214_0301
DWCC found
DWCC found
DWCC found
DCC found
DCC found
DCC found
DWCC found
DWCC found
DCC found
DCC found
DCC found
DCC found
DCC found


<ipython-input-1-af2496389471>:333: RuntimeWarning: Mean of empty slice.
  up_dcc_mean[group_num].append(dcc_ws[dcc_ws>0].mean())


In [31]:
draw_ellipses(date, reflect, rrate, lon, lat, z, ter, ell, dcc_ell, dwcc_ell, wcc_ell, bsr_ell, up5_x, up5_y)

/usr/local/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-29-bd8c59b589be>:14: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("HomeyerRainbow"))
  cs1.cmap.set_under('w')


In [ ]:
#lons = (lon[locy,locx])
#lats = (lat[locy,locx])

lons = (lon[up5_y[68],up5_x[68]])
lats = (lat[up5_y[68],up5_x[68]])

print (np.shape(lons))

In [ ]:
plt.figure(figsize=(5,6))
plt.contour(lon, lat, ter, levels=[500])
plt.contourf(lon, lat, reflect[0,:,:], levels=np.arange(0, 71, 2), cmap='pyart_HomeyerRainbow')
plt.scatter(lons, lats, color='k', s=8)


In [ ]:
# # ## Here is how to write out to csv:

# # dic= ({'loc':loc,
# #           'area':pf_area,
# #           'convective_area':carea,
# #           'stratiform_area':sarea,
# #           'mean_dbz_by_altitude':mean_dbz_z,
# #           'max_dbz_by_altitude':max_dbz_z,
# #           'echo_top': et,
# #           '40dbz_top': et40,
# #           'max_updraft': upmax,
# #           'height_of_max_updraft': upmax_alt,
# #           '5m/s_updraft_volume': upvol5,
# #           '10m/s_updraft_volume': upvol10,
# #           'mean_rainrate_from_RAINNC': rr_mean,
# #           'max_rainrate_from_RAINNC': rr_max,
# #           'max_convective_rainrate_from_RAINNC': crr_mean,
# #           'max_stratiform_rainrate_from_RAINNC': srr_mean,
# #           'dcc_xloc': dcc_x,
# #           'dcc_yloc': dcc_y,
# #           'dcc_area': dcc_ar,
# #           'dcc_dbz_at_10km': dcc_10km,
# #           'dcc_max_updraft': dcc_wmax,
# #           'dcc_mean_updraft': dcc_wmean,
# #           'dwcc_xloc': dwcc_x,
# #           'dwcc_yloc': dwcc_y,
# #           'dwcc_area': dwcc_ar,
# #           'dwcc_dbz_at_10km': dwcc_10km,
# #           'dwcc_max_updraft': dwcc_wmax,
# #           'dwcc_mean_updraft': dwcc_wmean,
# #           'wcc_xloc': wcc_x,
# #           'wcc_yloc': wcc_y,
# #           'wcc_area': wcc_ar,
# #           'wcc_max_updraft': wcc_wmax,
# #           'wcc_mean_updraft': wcc_wmean,
# #           'bsr_xloc': bsr_x,
# #           'bsr_yloc': bsr_y,
# #           'bsr_area': bsr_ar})
       
# # import pandas as pd 
# # df= pd.DataFrame(dic)
    
# # ind = np.where(df['echo_top'] > 0)
# # out1 = df.iloc[ind[0]] 
# # out1['time'] = date
    
# # out1.to_csv("wrf_relampago_test_1km_refl10cm_trmm_"+date+".csv", 
# #             columns=['time', 'loc', 'area', 'convective_area', 'stratiform_area', 'mean_dbz_by_altitude',
# #                      'max_dbz_by_altitude', 'echo_top', '40dbz_top', 'max_updraft', 'height_of_max_updraft',
# #                      '5m/s_updraft_volume', '10m/s_updraft_volume', 'mean_rainrate_from_RAINNC', 
# #                      'max_rainrate_from_RAINNC', 'max_convective_rainrate_from_RAINNC', 'max_stratiform_rainrate_from_RAINNC',
# #                      'dcc_xloc', 'dcc_yloc', 'dcc_area', 'dcc_dbz_at_10km', 'dcc_max_updraft',
# #                      'dcc_mean_updraft', 'dwcc_xloc', 'dwcc_yloc', 'dwcc_area', 'dwcc_dbz_at_10km',
# #                      'dwcc_max_updraft', 'dwcc_mean_updraft', 'wcc_xloc', 'wcc_yloc', 'wcc_area',
# #                      'wcc_max_updraft', 'wcc_mean_updraft', 'bsr_xloc', 'bsr_yloc', 'bsr_area'])
                     
                    

In [ ]:
draw_ellipses(date, reflect, rrate, lon, lat, z, ter, ell, dcc_ell, dwcc_ell, wcc_ell, bsr_ell, up5_ell)


In [ ]:
plt.figure(figsize=(16,4))
ax1 = plt.subplot(131)
cs1 = ax1.pcolormesh(lon, lat, rrate, cmap='pyart_HomeyerRainbow', vmin=0.001, vmax=30)
cs1.cmap.set_under('w')
plt.colorbar(cs1)
ax1.contour(lon, lat, reflect[0,:,:], levels=[0], colors='k')
ax1.set_title('WRF RAINNC (mm in 10 minute)')


ax2 = plt.subplot(132)
cs1 = ax2.pcolormesh(lon, lat, np.array(rain_zr)/6, cmap='pyart_HomeyerRainbow', vmin=0.001, vmax=30)
cs1.cmap.set_under('w')
plt.colorbar(cs1)
ax2.contour(lon, lat, reflect[0,:,:], levels=[0], colors='k')
ax2.set_title('Z-R at 2.5 km MSL (mm/hr divided by 6)')


ax3 = plt.subplot(133)
cs1 = ax3.pcolormesh(lon, lat, np.array(rrate) - np.array(rain_zr)/6, cmap='seismic', vmin=-10, vmax=10)
plt.colorbar(cs1)
ax3.contour(lon, lat, reflect[0,:,:], levels=[0], colors='k')
ax3.set_title('WRF - Z-R difference')

plt.suptitle('Rain rates at 20181214_0400', y=1.05)

#plt.savefig('rr_comp_test_pfellipses_20181214_0400.png')

In [ ]:
plt.figure(figsize=(12,6))
ax1 = plt.subplot(211)
t = np.arange(0, 113, 1)
#plt.plot(t[50:101], np.array(rr_max_wrf[50:101]), label='from WRF RAINNC')

#plt.plot(t[50:101], np.array(rr_max_zr[50:101])/60, label='from Z-R DROPS2.0')


ax1.plot(t[:], (np.array(rr_max_wrf[:]) - np.array(rr_max_zr[:])/6), label='diff, wrf - zr/6', lw=3)

ax1.axhline(0, 200, 0, color='k', lw=1)
plt.ylim(-10, 20)
ax1.set_xlim(0, 112)
#ax1.legend(loc=1, fontsize=12)
ax1.set_ylabel('rain rate difference\n[mm/ 10 min]', fontsize=14, labelpad=25)
#ax1.set_xlabel('PF number', fontsize=14, labelpad=25)
ax1.set_title('Rain rate difference between RAINNC and Z-R at 20181214_0200', fontsize=17)

ax2 = plt.subplot(212)
ax2.plot(t[:], np.array(pf_area)/1000, lw=3)
ax2.set_xlim(0, 112)
ax2.set_ylim(0, 110)
ax2.set_ylabel('area [10^3 km^2]', fontsize=14, labelpad=25)
ax2.set_xlabel('PF number', fontsize=14, labelpad=25)
ax2.set_title('PF area at 20181214_0200', fontsize=17)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.4)

#plt.savefig('rr_diff_test_pfellipses_20181214_0200.png')